In [3]:
import os
import time
import timm
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR
from PIL import Image
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from torchsummary import summary
import gc

In [4]:
gc.collect()
torch.cuda.empty_cache()

In [5]:
# 데이터셋 클래스 정의
class ImageDataset(Dataset):
    def __init__(self, csv, path, transform=None):
        """
        데이터셋을 초기화합니다.

        Args:
            csv (str 또는 pd.DataFrame): 이미지 파일 경로와 레이블이 포함된 CSV 파일 경로 또는 DataFrame.
            path (str): 이미지 파일들이 저장된 디렉토리의 경로.
            transform (callable, optional): 이미지에 적용할 변환 함수. 기본값은 None.
        """
        if isinstance(csv, pd.DataFrame):
            # CSV가 DataFrame일 경우, 값을 배열로 변환
            self.df = csv.values
        else:
            # CSV 파일을 읽어서 값을 배열로 변환
            self.df = pd.read_csv(csv).values
        self.path = path
        self.transform = transform

    def __len__(self):
        """
        데이터셋의 총 샘플 수를 반환합니다.

        Returns:
            int: 데이터셋의 샘플 수
        """
        return len(self.df)
    
    def __getitem__(self, idx):
        """
        주어진 인덱스에 해당하는 샘플을 반환합니다.

        Args:
            idx (int): 데이터셋에서 샘플의 인덱스.

        Returns:
            tuple: (이미지, 레이블)
        """
        name, target = self.df[idx]  # DataFrame에서 이미지 파일명과 타겟 레이블 추출
        img_path = os.path.join(self.path, name)  # 이미지 파일 경로 생성
        if not os.path.exists(img_path):
            # 이미지 파일이 존재하지 않을 경우, 경고 메시지 출력 및 빈 이미지 생성
            print(f"Warning: Image not found: {img_path}")
            img = np.zeros((380, 380, 3), dtype=np.uint8)  # 기본 이미지 크기 및 채널 (RGB) 설정
        else:
            # 이미지 파일을 열고 RGB로 변환
            img = np.array(Image.open(img_path).convert('RGB'))

        if self.transform:
            # 변환 함수가 제공되면 이미지를 변환
            img = self.transform(image=img)['image']
        
        return img, target

In [7]:
# 학습 함수 정의
def train_one_epoch(loader, model, optimizer, loss_fn, device):
    """
    한 에포크 동안 모델을 학습합니다.

    Args:
        loader (DataLoader): 학습 데이터 로더.
        model (nn.Module): 학습할 PyTorch 모델.
        optimizer (torch.optim.Optimizer): 모델 파라미터를 업데이트할 옵티마이저.
        loss_fn (callable): 손실 함수.
        device (torch.device): 모델과 데이터를 배치할 디바이스 (CPU 또는 GPU).

    Returns:
        tuple: 평균 손실, 정확도, F1 점수
    """
    model.train()  # 모델을 학습 모드로 설정
    train_loss = 0  # 에포크 동안의 총 손실을 저장할 변수
    preds_list = []  # 예측값을 저장할 리스트
    targets_list = []  # 실제 레이블을 저장할 리스트

    # 데이터 로더를 순회하며 학습
    pbar = tqdm(loader, desc="Training")  # 진행 상황을 표시하는 프로그래스 바

    for image, targets in pbar:
        image = image.to(device)  # 입력 이미지를 디바이스로 이동
        targets = targets.to(device)  # 타겟 레이블을 디바이스로 이동

        optimizer.zero_grad()  # 기울기를 초기화

        preds = model(image)  # 모델을 사용하여 예측값 계산
        loss = loss_fn(preds, targets)  # 손실 함수 계산
        loss.backward()  # 역전파를 통해 기울기 계산
        optimizer.step()  # 옵티마이저를 사용하여 모델 파라미터 업데이트

        train_loss += loss.item()  # 손실 값을 누적
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())  # 예측값을 CPU로 이동하여 리스트에 추가
        targets_list.extend(targets.detach().cpu().numpy())  # 실제 레이블을 CPU로 이동하여 리스트에 추가

        pbar.set_description(f"Loss: {loss.item():.4f}")  # 진행 상황에 현재 손실 값을 표시

    train_loss /= len(loader)  # 에포크 동안의 평균 손실 계산
    train_acc = accuracy_score(targets_list, preds_list)  # 정확도 계산
    train_f1 = f1_score(targets_list, preds_list, average='macro')  # F1 점수 계산

    return train_loss, train_acc, train_f1  # 평균 손실, 정확도, F1 점수를 반환


In [8]:
# 검증 함수 정의
def validate(loader, model, loss_fn, device):
    """
    모델의 검증을 수행합니다.

    Args:
        loader (DataLoader): 검증 데이터 로더.
        model (nn.Module): 검증할 PyTorch 모델.
        loss_fn (callable): 손실 함수.
        device (torch.device): 모델과 데이터를 배치할 디바이스 (CPU 또는 GPU).

    Returns:
        tuple: 평균 손실, 정확도, F1 점수
    """
    model.eval()  # 모델을 평가 모드로 설정
    val_loss = 0  # 검증 동안의 총 손실을 저장할 변수
    preds_list = []  # 예측값을 저장할 리스트
    targets_list = []  # 실제 레이블을 저장할 리스트

    with torch.no_grad():  # 검증 중에는 기울기 계산을 하지 않음
        for image, targets in loader:
            image = image.to(device)  # 입력 이미지를 디바이스로 이동
            targets = targets.to(device)  # 타겟 레이블을 디바이스로 이동

            preds = model(image)  # 모델을 사용하여 예측값 계산
            loss = loss_fn(preds, targets)  # 손실 함수 계산

            val_loss += loss.item()  # 손실 값을 누적
            preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())  # 예측값을 CPU로 이동하여 리스트에 추가
            targets_list.extend(targets.detach().cpu().numpy())  # 실제 레이블을 CPU로 이동하여 리스트에 추가

    val_loss /= len(loader)  # 검증 동안의 평균 손실 계산
    val_acc = accuracy_score(targets_list, preds_list)  # 정확도 계산
    val_f1 = f1_score(targets_list, preds_list, average='macro')  # F1 점수 계산

    return val_loss, val_acc, val_f1  # 평균 손실, 정확도, F1 점수를 반환

In [9]:
# 모델 구조 출력 함수
def print_model_summary(model, input_size):
    summary(model, input_size)

In [15]:
# 메인 실행 코드
if __name__ == "__main__":
    # 설정
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    data_path = '/data/ephemeral/home/data/'
    model_name = 'efficientnet_b4'
    img_size = 380  # EfficientNet-B4에 적합한 이미지 크기
    LR = 5e-4  # 학습률 조정
    EPOCHS = 30
    BATCH_SIZE = 32  # 배치 크기 감소
    num_workers = 4

    # 데이터 증강 설정
    train_transform = A.Compose([
        A.Resize(height=img_size, width=img_size),  # 이미지 크기 조정
        A.HorizontalFlip(p=0.5),  # 확률적으로 수평 플립
        A.VerticalFlip(p=0.5),  # 확률적으로 수직 플립
        A.RandomRotate90(p=0.5),  # 확률적으로 90도 회전
        A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.5),  # 이동, 크기 조정 및 회전
        A.RandomBrightnessContrast(p=0.5),  # 확률적으로 밝기 및 대비 조정
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # 이미지 정규화
        ToTensorV2(),  # 이미지 텐서로 변환
    ])

    val_transform = A.Compose([
        A.Resize(height=img_size, width=img_size),  # 이미지 크기 조정
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # 이미지 정규화
        ToTensorV2(),  # 이미지 텐서로 변환
    ])

    # 데이터 로드 및 분할
    df = pd.read_csv(os.path.join(data_path, "train_add.csv"))
    train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['target'])

    # 데이터셋 객체를 생성
    train_dataset = ImageDataset(train_df, os.path.join(data_path, "train_add"), transform=train_transform)
    val_dataset = ImageDataset(val_df, os.path.join(data_path, "train_add"), transform=val_transform)
    test_dataset = ImageDataset(os.path.join(data_path, "sample_submission.csv"), os.path.join(data_path, "test"), transform=val_transform)

    # 데이터 로더 객체를 생성
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=num_workers, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=num_workers, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=num_workers, pin_memory=True)

    # 모델 설정
    model = timm.create_model(model_name, pretrained=True, num_classes=17).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = AdamW(model.parameters(), lr=LR, weight_decay=1e-5)
    scheduler = CosineAnnealingLR(optimizer, T_max=EPOCHS)

    # 모델 구조 출력
    print(f"\nModel structure of {model_name}:")
    print_model_summary(model, (3, img_size, img_size))

    # 학습 루프
    best_val_f1 = 0
    for epoch in range(EPOCHS):
        train_loss, train_acc, train_f1 = train_one_epoch(train_loader, model, optimizer, loss_fn, device)
        val_loss, val_acc, val_f1 = validate(val_loader, model, loss_fn, device)
        scheduler.step()

        print(f"Epoch {epoch+1}/{EPOCHS}")
        print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Train F1: {train_f1:.4f}")
        print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, Val F1: {val_f1:.4f}")

        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            torch.save(model.state_dict(), "best_model.pth")

    # 테스트 데이터 추론
    model.load_state_dict(torch.load("best_model.pth"))
    model.eval()
    preds_list = []

    for image, _ in tqdm(test_loader):
        image = image.to(device)
        with torch.no_grad():
            preds = model(image)
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())

    # 결과 저장
    pred_df = pd.DataFrame(test_dataset.df, columns=['ID', 'target'])
    pred_df['target'] = preds_list
    pred_df.to_csv("pred_efficientNetB4.csv", index=False)
    print("Prediction completed and saved to pred.csv")


Model structure of efficientnet_b4:
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 48, 190, 190]           1,296
          Identity-2         [-1, 48, 190, 190]               0
              SiLU-3         [-1, 48, 190, 190]               0
    BatchNormAct2d-4         [-1, 48, 190, 190]              96
            Conv2d-5         [-1, 48, 190, 190]             432
          Identity-6         [-1, 48, 190, 190]               0
              SiLU-7         [-1, 48, 190, 190]               0
    BatchNormAct2d-8         [-1, 48, 190, 190]              96
            Conv2d-9             [-1, 12, 1, 1]             588
             SiLU-10             [-1, 12, 1, 1]               0
           Conv2d-11             [-1, 48, 1, 1]             624
          Sigmoid-12             [-1, 48, 1, 1]               0
    SqueezeExcite-13         [-1, 48, 190, 190]               0
  

Loss: 0.5300: 100%|██████████| 44/44 [00:13<00:00,  3.15it/s]


Epoch 1/30
Train Loss: 1.3605, Train Acc: 0.5970, Train F1: 0.5941
Val Loss: 0.3334, Val Acc: 0.8963, Val F1: 0.8970


Loss: 0.1011: 100%|██████████| 44/44 [00:13<00:00,  3.21it/s]


Epoch 2/30
Train Loss: 0.3284, Train Acc: 0.8861, Train F1: 0.8868
Val Loss: 0.3426, Val Acc: 0.8703, Val F1: 0.8629


...

Epoch 29/30
Train Loss: 0.0027, Train Acc: 0.9993, Train F1: 0.9993
Val Loss: 0.2287, Val Acc: 0.9452, Val F1: 0.9465


Loss: 0.0017: 100%|██████████| 44/44 [00:13<00:00,  3.18it/s]


Epoch 30/30
Train Loss: 0.0033, Train Acc: 0.9993, Train F1: 0.9993
Val Loss: 0.2339, Val Acc: 0.9452, Val F1: 0.9462


100%|██████████| 99/99 [00:08<00:00, 11.70it/s]

Prediction completed and saved to pred.csv


In [17]:
from sklearn.model_selection import KFold

# 데이터 로드
data_path = '/data/ephemeral/home/data/'
df = pd.read_csv(os.path.join(data_path, "train_add.csv"))

# K-Fold 설정
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

In [18]:
def get_data_loaders(df, data_path, fold, kfold, batch_size, num_workers):
    train_idx, val_idx = list(kfold.split(df))[fold]
    train_df = df.iloc[train_idx]
    val_df = df.iloc[val_idx]
    
    train_transform = A.Compose([
        A.Resize(height=img_size, width=img_size),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.RandomRotate90(p=0.5),
        A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=20, p=0.5),
        A.RandomBrightnessContrast(p=0.5),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])

    val_transform = A.Compose([
        A.Resize(height=img_size, width=img_size),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])

    train_dataset = ImageDataset(train_df, os.path.join(data_path, "train_add"), transform=train_transform)
    val_dataset = ImageDataset(val_df, os.path.join(data_path, "train_add"), transform=val_transform)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)
    
    return train_loader, val_loader


In [19]:
def train_one_epoch(train_loader, model, optimizer, loss_fn, device):
    model.train()
    epoch_loss = 0
    correct = 0
    total = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    return epoch_loss / len(train_loader), correct / total

def validate(val_loader, model, loss_fn, device):
    model.eval()
    epoch_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = loss_fn(outputs, labels)
            
            epoch_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    return epoch_loss / len(val_loader), correct / total


In [20]:
EPOCHS = 30
BATCH_SIZE = 32
NUM_WORKERS = 4
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 하이퍼파라미터
LR = 5e-4
model_name = 'efficientnet_b4'
img_size = 380

# K-Fold 교차 검증 수행
for fold in range(5):
    print(f"Fold {fold+1}/{5}")
    
    train_loader, val_loader = get_data_loaders(df, data_path, fold, kfold, BATCH_SIZE, NUM_WORKERS)
    
    # 모델, 손실 함수, 최적화기 설정
    model = timm.create_model(model_name, pretrained=True, num_classes=17).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = AdamW(model.parameters(), lr=LR, weight_decay=1e-5)
    scheduler = CosineAnnealingLR(optimizer, T_max=EPOCHS)
    
    # 학습 루프
    for epoch in range(EPOCHS):
        train_loss, train_acc = train_one_epoch(train_loader, model, optimizer, loss_fn, device)
        val_loss, val_acc = validate(val_loader, model, loss_fn, device)
        scheduler.step()
        
        print(f"Epoch {epoch+1}/{EPOCHS}")
        print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
        print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
    
    # 모델 저장 (필요한 경우)
    torch.save(model.state_dict(), f"model_fold_{fold+1}.pth")


Fold 1/5
Epoch 1/30
Train Loss: 1.4633, Train Acc: 0.5775
Val Loss: 0.4290, Val Acc: 0.8588
Epoch 2/30
Train Loss: 0.3913, Train Acc: 0.8536
Val Loss: 0.3210, Val Acc: 0.8790
Epoch 3/30
Train Loss: 0.2544, Train Acc: 0.9120
Val Loss: 0.2606, Val Acc: 0.9049
Epoch 4/30
Train Loss: 0.1764, Train Acc: 0.9344
Val Loss: 0.2659, Val Acc: 0.9107
Epoch 5/30
Train Loss: 0.1160, Train Acc: 0.9560
Val Loss: 0.2936, Val Acc: 0.8963
Epoch 6/30
Train Loss: 0.1107, Train Acc: 0.9625
Val Loss: 0.2073, Val Acc: 0.9337
Epoch 7/30
Train Loss: 0.0887, Train Acc: 0.9719
Val Loss: 0.2133, Val Acc: 0.9222
Epoch 8/30
Train Loss: 0.0745, Train Acc: 0.9748
Val Loss: 0.1766, Val Acc: 0.9280
Epoch 9/30
Train Loss: 0.0452, Train Acc: 0.9877
Val Loss: 0.2504, Val Acc: 0.9193
Epoch 10/30
Train Loss: 0.0477, Train Acc: 0.9834
Val Loss: 0.2085, Val Acc: 0.9280
Epoch 11/30
Train Loss: 0.0369, Train Acc: 0.9885
Val Loss: 0.2213, Val Acc: 0.9280
Epoch 12/30
Train Loss: 0.0405, Train Acc: 0.9870
Val Loss: 0.2473, Val Acc:

In [21]:
# 메인 실행 코드
if __name__ == "__main__":
    # 설정
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    data_path = '/data/ephemeral/home/data/'
    model_name = 'efficientnet_b4'
    img_size = 380  # EfficientNet-B4에 적합한 이미지 크기
    BATCH_SIZE = 32  # 배치 크기
    num_workers = 4

    # 데이터 증강 설정
    val_transform = A.Compose([
        A.Resize(height=img_size, width=img_size),  # 이미지 크기 조정
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # 이미지 정규화
        ToTensorV2(),  # 이미지 텐서로 변환
    ])

    # 테스트 데이터 로드
    test_dataset = ImageDataset(os.path.join(data_path, "sample_submission.csv"), os.path.join(data_path, "test"), transform=val_transform)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=num_workers, pin_memory=True)

    # 모델 설정
    model = timm.create_model(model_name, pretrained=False, num_classes=17).to(device)
    model.load_state_dict(torch.load('/data/ephemeral/home/code/BeomCheol_code/best_model.pth'))  # 업로드된 모델 파일 로드
    model.eval()

    # 테스트 데이터 추론
    preds_list = []
    for image, _ in tqdm(test_loader):
        image = image.to(device)
        with torch.no_grad():
            preds = model(image)
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())

    # 결과 저장
    pred_df = pd.DataFrame(test_dataset.df, columns=['ID', 'target'])
    pred_df['target'] = preds_list
    pred_df.to_csv("pred_efficientNetB4_kFold.csv", index=False)
    print("Prediction completed and saved to pred_efficientNetB4.csv")

100%|██████████| 99/99 [00:08<00:00, 11.88it/s]

Prediction completed and saved to pred_efficientNetB4.csv
